In [1]:
import pandas as pd
import numpy as np
import json
from log_parser import process_file

In [51]:
_fname = "../replays/log4.log"
_time_step = 100

In [52]:
# load data from file

obj = process_file(_fname)
df = pd.DataFrame(obj)

listings = pd.DataFrame(df["listings"][0])
df = df.drop("listings", axis=1)

Reading from input file ../replays/log4.log


In [53]:
def flatten_trades(df, col, is_me):
    # get market trades

    data = []
    for index, row  in df.iterrows():
        all_trades = list(row[col].values())
        for sym_trades in all_trades:
            for trade in sym_trades:
                trade["fill_time"] = row["timestamp"]
            data += sym_trades

    df = pd.DataFrame(data)
    df = df.rename({"timestamp": "order_time"}, axis=1)
    
    # calculate info about my trades
    df["is_me"] = is_me
    df["my_action"] = np.where(df["buyer"] == "SUBMISSION", "buy", None)
    df["my_action"] = np.where(df["seller"] == "SUBMISSION", "sell", None)
    df["my_quantity"] = df["quantity"] * df["is_me"].astype(int) * (2 * (df["my_action"] == "buy").astype(int) - 1)
    
    return df

In [60]:
market_trades = flatten_trades(
    df, 
    "market_trades", 
    is_me=False
)

my_trades = flatten_trades(
    df, 
    "own_trades", 
    is_me=True
).sort_values(by="fill_time")


# filter market trades (only keep those where order_time = fill_time)
market_trades = market_trades[market_trades["fill_time"] == market_trades["order_time"]]

# filter my_trades (only keep those where fill_time = order_time + 100)
my_trades = my_trades[my_trades["fill_time"] == my_trades["order_time"] + _time_step]


trade_df = pd.concat([market_trades, my_trades])
trade_df = trade_df.sort_values(by="fill_time").reset_index(drop=True)

my_trades = trade_df[trade_df["is_me"]]
market_trades = trade_df[~trade_df["is_me"]]


In [61]:
my_trades.iloc[:20]

,buyer,price,quantity,seller,symbol,order_time,fill_time,is_me,my_action,my_quantity
1,,9998.0,1,SUBMISSION,PEARLS,0,100,True,sell,-1
2,,9996.0,1,SUBMISSION,PEARLS,100,200,True,sell,-1
3,,9996.0,1,SUBMISSION,PEARLS,200,300,True,sell,-1
4,,9996.0,2,SUBMISSION,PEARLS,300,400,True,sell,-2
5,,9998.0,5,SUBMISSION,PEARLS,400,500,True,sell,-5
6,,10002.0,1,SUBMISSION,PEARLS,500,600,True,sell,-1
7,,9996.0,2,SUBMISSION,PEARLS,600,700,True,sell,-2
8,,9998.0,7,SUBMISSION,PEARLS,700,800,True,sell,-7


In [62]:
market_trades

,buyer,price,quantity,seller,symbol,order_time,fill_time,is_me,my_action,my_quantity
0,,10002.0,1,,PEARLS,100,100,False,None,0
9,,4891.0,1,,BANANAS,1400,1400,False,None,0
10,,9998.0,5,,PEARLS,2700,2700,False,None,0
11,,9998.0,1,,PEARLS,2900,2900,False,None,0
12,,4891.0,1,,BANANAS,3100,3100,False,None,0
...,...,...,...,...,...,...,...,...,...,...
298,,4903.0,1,,BANANAS,197700,197700,False,None,0
299,,9998.0,1,,PEARLS,198400,198400,False,None,0
300,,10002.0,3,,PEARLS,198500,198500,False,None,0
301,,4910.0,1,,BANANAS,198900,198900,False,None,0


In [63]:
df["market_trades"][1997]

{'BANANAS': [{'buyer': '',
   'price': 4906.0,
   'quantity': 3,
   'seller': '',
   'symbol': 'BANANAS',
   'timestamp': 199900,
   'fill_time': 199900}],
 'PEARLS': [{'buyer': '',
   'price': 10002.0,
   'quantity': 1,
   'seller': '',
   'symbol': 'PEARLS',
   'timestamp': 199700,
   'fill_time': 199900}]}